In [1]:
import json
import pandas as pd
from datasets import Dataset

def load_data(file_path):
    """
    JSON 데이터를 불러와 Pandas DataFrame으로 변환
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        json_data = json.load(f)
        
    data = []
    for item in json_data['data']:
        if 'text' in item and 'keyword' in item:
            data.append({
                'text': item['text'],
                'keyword': item['keyword']
            })
    
    df = pd.DataFrame(data)
    print(df.info())
    # None 값을 빈 문자열로 대체
    df = df.fillna('')
    return df


# JSON 파일 경로
file_path_smell = "/home/yjtech2/Desktop/yurim/LLM/Data/smell_keyword/smell_type_data.json"
smell_df = load_data(file_path_smell)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   text     30000 non-null  object
 1   keyword  30000 non-null  object
dtypes: object(2)
memory usage: 468.9+ KB
None


In [2]:
from sklearn.model_selection import train_test_split

# 데이터를 70:30 비율로 나누기
def split_data(df, train_ratio = 0.8):
    """
    DataFrame을 train과 val로 나눔
    """
    train_df, val_df = train_test_split(df, train_size = train_ratio, random_state = 42, shuffle = True)
    return train_df, val_df

# train, val로 나누기
train_smell_df, val_smell_df = split_data(smell_df)

train_smell_df.to_csv('/home/yjtech2/Desktop/yurim/LLM/Data/smell_keyword/train_smell_df.csv', index = False)
val_smell_df.to_csv('/home/yjtech2/Desktop/yurim/LLM/Data/smell_keyword/val_smell_df.csv', index = False)


print('Smell')
print("Train Data:")
print(train_smell_df.head())
print("\nValidation Data:")
print(val_smell_df.head())

# 데이터 크기 확인
print(f"Train size: {len(train_smell_df)}, Validation size: {len(val_smell_df)}")


Smell
Train Data:
                                                    text          keyword
21753  여보세요? 택배 물류 창고 근처에서 12개월 전 새벽 5시에 화장실 냄새 청결 부족...           화장실 냄새
251    안녕하세요. 지하철 역 주변에서 11월 14일 오전에 공공장소 쓰레기 냄새 관련하여...      공공장소 쓰레기 냄새
22941  안녕하세요. 아파트 커뮤니티 센터에서 12월 8일 저녁 7시에 오래된 냄새 관련하여...           오래된 냄새
618    여보세요? 음식물 쓰레기장에서 1월 17일 자정에 발생한 썩은 물질 냄새 불법 폐기...         썩은 물질 냄새
17090  여보세요? 돼지 농장에서 5월 4일 저녁 7시에 발생한 동물 건강 관리용 약물 냄새...  동물 건강 관리용 약물 냄새

Validation Data:
                                                    text        keyword
2308   여보세요? 농업 생산 기지에서 1년 전 저녁에 발생한 강한 페인트 잔여물 냄새 쓰레...  강한 페인트 잔여물 냄새
22404  안녕하세요. 최근 하수 처리장 근처에서 9월 12일 오전 11시부터 12시에 발생한...       썩은 음식 냄새
23397  안녕하세요. 거리 공연장 근처에서 11월 28일 오후 8시부터 9시에 발생한 비린 ...       비린 고기 냄새
25058  여보세요? 대형 교량 공사 현장에서 11월 10일 저녁 6시에 발생한 구리 배선 냄...       구리 배선 냄새
2664   안녕하세요. 최근 고속도로 톨게이트 주변에서 7개월 전 자정에 발생한 부패된 유기물...     부패된 유기물 냄새
Train size: 24000, Validation size: 6000


In [3]:
import pandas as pd
from datasets import Dataset

train_smell_df = pd.read_csv('/home/yjtech2/Desktop/yurim/LLM/Data/smell_keyword/train_smell_df.csv')
val_smell_df = pd.read_csv('/home/yjtech2/Desktop/yurim/LLM/Data/smell_keyword/val_smell_df.csv')



train_smell_data_dict = train_smell_df.to_dict(orient='list')
smell_train_dataset = Dataset.from_dict(train_smell_data_dict)
smell_train_dataset
print(len(smell_train_dataset))

val_smell_data_dict = val_smell_df.to_dict(orient='list')
smell_val_dataset = Dataset.from_dict(val_smell_data_dict)
print(len(smell_val_dataset))

24000
6000


In [4]:
import os
import torch
from tqdm import tqdm
from typing import Dict
import time
from datetime import datetime
import numpy as np
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    AdamW,
    DataCollatorForSeq2Seq,
)

from transformers import get_linear_schedule_with_warmup
from torch.cuda.amp import GradScaler, autocast
from mecab import MeCab

class CustomKeyBERTTrainer:
    def __init__(self, model_name: str, **kwargs):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"Using device: {self.device}")
        if self.device == "cuda":
            print(f"GPU Model: {torch.cuda.get_device_name(0)}")
            print(f"Available GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
        
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(self.device)
        self.optimizer = AdamW(self.model.parameters(), lr=kwargs.get("learning_rate", 1e-4))
        self.max_length = kwargs.get("max_length", 512)
        self.training_args = kwargs
        self.save_dir = kwargs.get("save_dir", "./smell_models")
        self.mecab = MeCab()  # MeCab 초기화
        self.best_model_path = os.path.join(self.save_dir, "pytorch_model.bin")
        self.tokenizer_path = self.save_dir
        
        self.history = {
            'train_loss': [],
            'val_loss': [],
            'epoch_times': [],
            'best_epoch': 0
        }

        os.makedirs(self.save_dir, exist_ok=True)
        
    def _mecab_tokenize(self, text: str) -> str:
        """
        MeCab을 사용하여 입력 텍스트에서 명사, 형용사, 부사를 추출
        """
        pos_tags = self.mecab.pos(text)  # 품사 태그 추출
        # 원하는 품사 필터링: 명사(NNG, NNP), 형용사(VA), 부사(MAG, MAJ)
        keywords = [
            word for word, pos in pos_tags if pos in ("NNG", "NNP", "VA", "MAG", "MAJ")
        ]
        return " ".join(keywords)  # 추출한 단어를 공백으로 연결하여 반환
    
    def preprocess_data(self, examples: Dict) -> Dict:
        # MeCab 형태소 분석과 품사 필터링 적용
        examples["text"] = [self._mecab_tokenize(text) for text in examples["text"]]
        
        # 기존 코드 유지
        inputs = [f"키워드 추출: {text}" for text in examples["text"]]
        model_inputs = self.tokenizer(
            inputs,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors=None  # 텐서 변환을 DataCollator에 맡김
        )

        # 레이블(키워드) 처리
        # 키워드가 리스트가 아니라 단일 문자열일 경우에도 처리 가능하도록 수정
        labels = []
        for keyword in examples["keyword"]:
            if isinstance(keyword, list):  # 리스트일 경우
                labels.append(", ".join(keyword) if keyword else "")
            else:  # 단일 문자열일 경우
                labels.append(keyword if keyword else "")
        
        with self.tokenizer.as_target_tokenizer():
            tokenized_labels = self.tokenizer(
                labels,
                max_length=self.max_length,
                padding="max_length",
                truncation=True,
                return_tensors=None  # 텐서 변환을 DataCollator에 맡김
            )

        # -100으로 패딩 토큰을 마스킹
        labels = tokenized_labels["input_ids"]
        for i in range(len(labels)):
            for j in range(len(labels[i])):
                if labels[i][j] == self.tokenizer.pad_token_id:
                    labels[i][j] = -100

        model_inputs["labels"] = labels
        return model_inputs
        
    def save_model_and_tokenizer(self, epoch=None, is_best=False):
        """
        최고 성능 모델만 저장하고 이전 모델을 삭제
        """
        if is_best:
            # 이전 최고 모델 디렉토리 삭제
            if os.path.exists(self.best_model_path):
                print(f"Deleting previous best model at {self.best_model_path}")
                os.system(f"rm -rf {self.best_model_path}")
            
            # 새로운 최고 모델 저장
            save_path = os.path.join(self.save_dir, f"best_model_epoch_{epoch}")
            os.makedirs(save_path, exist_ok=True)
            self.model.save_pretrained(save_path)
            self.tokenizer.save_pretrained(save_path)
            torch.save(self.history, os.path.join(save_path, 'training_history.pt'))
            print(f"New best model saved at {save_path}")

            # 최고 모델 경로 업데이트
            self.best_model_path = save_path

    def calculate_metrics(self, predictions, labels):
        predictions = torch.argmax(predictions, dim=-1)
        correct = (predictions == labels).masked_fill(labels == -100, 0)
        accuracy = correct.sum().item() / (labels != -100).sum().item()
        return accuracy

    def train(self, train_dataset, valid_dataset=None):
        start_time = time.time()
        print(f"\nStarting training at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"Training parameters:")
        print(f"- Batch size: {self.training_args['batch_size']}")
        print(f"- Learning rate: {self.training_args.get('learning_rate', '1e-4')}")
        print(f"- Max length: {self.max_length}")
        print(f"- Number of epochs: {self.training_args['num_epochs']}")
        print(f"- Training samples: {len(train_dataset)}")
        if valid_dataset:
            print(f"- Validation samples: {len(valid_dataset)}")
        print("\n" + "="*50 + "\n")

        # 데이터셋 전처리
        print("Preprocessing training data...")
        train_dataset = train_dataset.map(
            self.preprocess_data,
            batched=True,
            remove_columns=train_dataset.column_names,
            desc="Processing training data"
        )

        if valid_dataset is not None:
            print("Preprocessing validation data...")
            valid_dataset = valid_dataset.map(
                self.preprocess_data,
                batched=True,
                remove_columns=valid_dataset.column_names,
                desc="Processing validation data"
            )

        # DataCollator 설정
        data_collator = DataCollatorForSeq2Seq(
            tokenizer=self.tokenizer,
            model=self.model,
            padding=True,
            return_tensors="pt"
        )

        # DataLoader 설정 (num_workers=0으로 변경하여 멀티프로세싱 관련 오류 방지)
        train_dataloader = torch.utils.data.DataLoader(
            train_dataset,
            batch_size=self.training_args["batch_size"],
            shuffle=True,
            collate_fn=data_collator,
            num_workers=0,
            pin_memory=True
        )

        if valid_dataset is not None:
            valid_dataloader = torch.utils.data.DataLoader(
                valid_dataset,
                batch_size=self.training_args["batch_size"],
                shuffle=False,
                collate_fn=data_collator,
                num_workers=0,
                pin_memory=True
            )
        best_val_loss = float('inf')
        early_stopping_counter = 0
        early_stopping_patience = self.training_args.get('patience', 3)

        for epoch in range(self.training_args["num_epochs"]):
            epoch_start_time = time.time()
            
            self.model.train()
            epoch_loss = 0
            epoch_accuracy = 0
            train_steps = 0
            
            progress_bar = tqdm(train_dataloader, desc=f"Training Epoch {epoch + 1}")
            batch_losses = []
            batch_accuracies = []
            
            for batch_idx, batch in enumerate(progress_bar):
                input_ids = batch["input_ids"].to(self.device)
                attention_mask = batch["attention_mask"].to(self.device)
                labels = batch["labels"].to(self.device)

                outputs = self.model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    labels=labels,
                )
                
                loss = outputs.loss # 손실 계산
                accuracy = self.calculate_metrics(outputs.logits, labels)
                
                loss.backward()# 손실 역전파 
                self.optimizer.step() # 가중치 업데이트
                self.optimizer.zero_grad() # 그래디언트 초기화

                batch_losses.append(loss.item())
                batch_accuracies.append(accuracy)
                
                current_loss = np.mean(batch_losses[-100:])
                current_accuracy = np.mean(batch_accuracies[-100:])
                progress_bar.set_postfix({
                    'loss': f'{current_loss:.4f}',
                    'accuracy': f'{current_accuracy:.4f}',
                    'batch': f'{batch_idx + 1}/{len(train_dataloader)}'
                })

            avg_train_loss = np.mean(batch_losses)
            avg_train_accuracy = np.mean(batch_accuracies)

            if valid_dataset is not None:
                self.model.eval()
                val_losses = []
                val_accuracies = []

                print("\nRunning validation...")
                with torch.no_grad():
                    for batch in tqdm(valid_dataloader, desc="Validating"):
                        input_ids = batch["input_ids"].to(self.device)
                        attention_mask = batch["attention_mask"].to(self.device)
                        labels = batch["labels"].to(self.device)

                        outputs = self.model(
                            input_ids=input_ids,
                            attention_mask=attention_mask,
                            labels=labels,
                        )
                        
                        loss = outputs.loss
                        accuracy = self.calculate_metrics(outputs.logits, labels)
                        
                        val_losses.append(loss.item())
                        val_accuracies.append(accuracy)

                avg_val_loss = np.mean(val_losses)
                avg_val_accuracy = np.mean(val_accuracies)

                if avg_val_loss < best_val_loss:
                    best_val_loss = avg_val_loss
                    early_stopping_counter = 0
                    self.history['best_epoch'] = epoch + 1
                    print(f"\nNew best validation loss: {best_val_loss:.4f}")
                    self.save_model_and_tokenizer(epoch + 1, is_best=True)  # 최고 모델만 저장
                else:
                    early_stopping_counter += 1


            epoch_time = time.time() - epoch_start_time
            self.history['epoch_times'].append(epoch_time)
            self.history['train_loss'].append(avg_train_loss)
            if valid_dataset is not None:
                self.history['val_loss'].append(avg_val_loss)

            # Print epoch summary
            print(f"\nEpoch {epoch + 1} Summary:")
            print(f"Time taken: {epoch_time:.2f} seconds")
            print(f"Average training loss: {avg_train_loss:.4f}")
            print(f"Training accuracy: {avg_train_accuracy:.4f}")
            if valid_dataset is not None:
                print(f"Validation loss: {avg_val_loss:.4f}")
                print(f"Validation accuracy: {avg_val_accuracy:.4f}")
                print(f"Best validation loss so far: {best_val_loss:.4f}")
                print(f"Early stopping counter: {early_stopping_counter}/{early_stopping_patience}")

            if early_stopping_counter >= early_stopping_patience:
                print("\nEarly stopping triggered.")
                break
    def predict(self, text: str) -> str:
            """모델 추론"""
            try:
                max_length = self.max_length - 2  # Reserve space for special tokens
                chunks = [text[i:i + max_length] for i in range(0, len(text), max_length)]
                results = []
                
                for chunk in chunks:
                    inputs = self.tokenizer(
                        f"키워드 추출: {self._normalize_text(chunk)}",
                        return_tensors="pt",
                        max_length=self.max_length,
                        truncation=True,
                    ).to(self.device)

                    outputs = self.model.generate(
                        inputs["input_ids"], 
                        max_length=self.max_length, 
                        num_beams=5
                    )
                    
                    result = self.tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
                    if result:
                        results.append(result)
                
                return ", ".join(results).strip() or ""
            
            except Exception as e:
                print(f"Keyword extraction error: {str(e)}")
                return ""
    def _normalize_text(self, text: str) -> str:
        return text.strip()

2024-12-04 16:22:41.052332: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-04 16:22:41.076499: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-04 16:22:41.410090: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"  # 멀티프로세싱 경고 방지

trainer = CustomKeyBERTTrainer(
    model_name="facebook/bart-base", # t5-base     skt/kobart-base-v2
    max_length=512,
    learning_rate=1e-4,
    batch_size=8,
    num_epochs=10,
    gradient_accumulation_steps=8,
    patience=3 # 몇 에폭마다 체크포인트 저장할지
)

if __name__ == "__main__":
    torch.cuda.empty_cache()  # GPU 메모리 초기화
    trainer.train(smell_train_dataset, smell_val_dataset)

Using device: cuda
GPU Model: NVIDIA GeForce RTX 4080
Available GPU memory: 15.59 GB


/home/yjtech2/Desktop/yurim/anaconda3/envs/venv-llm/lib/python3.8/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Parameter 'function'=<bound method CustomKeyBERTTrainer.preprocess_data of <__main__.CustomKeyBERTTrainer object at 0x7f1c2c5a4400>> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.



Starting training at: 2024-12-04 16:22:44
Training parameters:
- Batch size: 8
- Learning rate: 0.0001
- Max length: 512
- Number of epochs: 10
- Training samples: 24000
- Validation samples: 6000


Preprocessing training data...


Processing training data:   0%|          | 0/24000 [00:00<?, ? examples/s]

/home/yjtech2/Desktop/yurim/anaconda3/envs/venv-llm/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


...


Epoch 5 Summary:
Time taken: 632.66 seconds
Average training loss: 0.0304
Training accuracy: 0.9895
Validation loss: 0.0244
Validation accuracy: 0.9917
Best validation loss so far: 0.0207
Early stopping counter: 2/3


Training Epoch 6: 100%|██████████| 3000/3000 [09:43<00:00,  5.14it/s, loss=0.0359, accuracy=0.9879, batch=3000/3000]



Running validation...


Validating: 100%|██████████| 750/750 [00:49<00:00, 15.26it/s]


Epoch 6 Summary:
Time taken: 632.85 seconds
Average training loss: 0.0294
Training accuracy: 0.9897
Validation loss: 0.0262
Validation accuracy: 0.9909
Best validation loss so far: 0.0207
Early stopping counter: 3/3

Early stopping triggered.


In [10]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

def test_model(text, saved_model_path, type):

    # 모델과 토크나이저 불러오기
    tokenizer = AutoTokenizer.from_pretrained(saved_model_path, local_files_only=True)
    model = AutoModelForSeq2SeqLM.from_pretrained(saved_model_path, local_files_only=True).to('cpu')
    
    inputs = tokenizer(
        f"키워드 추출: {text}",
        return_tensors="pt",
        max_length=128,
        truncation=True
    ).to("cpu")

    outputs = model.generate(
        inputs["input_ids"],
        max_length=128,
        num_beams=5,
        length_penalty=0.7,
        repetition_penalty=1.2,
        early_stopping=True
    )

    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("원래 문장: ", text)
    print(f"{type}키워드 추출: {result}")
    

text = '음식물 쓰레기 냄새가 납니다'

test_model(text, '/home/yjtech2/Desktop/yurim/LLM/Pre_processing/smell_keyword/smell_models/best_model_epoch_3', 'smell')

원래 문장:  음식물 쓰레기 냄새가 납니다
smell키워드 추출: 불쾌한 냄새


In [14]:
print(val_smell_df.iloc[100:110]['text'])
print(val_smell_df.iloc[100:110]['keyword'])

100    여보세요? 유휴 토지 개발 현장에서 10개월 후 오후 10시부터 11시에 발생한 절...
101    저희 말 농장에서 가축 사료 냄새 문제로 불편을 겪고 있습니다. 30일 전 저녁 6...
102    여보세요? 청소 장비 제조 공장에서 4월 27일 오전 7시부터 8시에 발생한 제품 ...
103    저희 알루미늄 압출 공장에서 1월 13일 정오에 포장지 처리 냄새 기계 오작동로 인...
104    안녕하세요. 최근 포항 해수욕장에서 12일 전 밤에 발생한 부패된 고기 냄새 문제로...
105    안녕하세요. 최근 가정용 전자기기 공장에서 2월 21일 오후 5시부터 6시에 발생한...
106    여보세요? 레스토랑 및 카페 거리에서 2월 5일 오후 10시부터 11시에 젖은 쓰레...
107    여보세요? 죽도초등학교 인근에서 3월 17일 오전에 얼음이 녹은 냄새 온도 급변로 ...
108    저희 공원 산책로 인근에서 8일 후 오후 4시부터 5시에 플라스틱 태운 냄새 불법 ...
109    여보세요? 도시 외곽 축사 단지에서 8월 1일 밤 11시에 발생한 동물 배설물 냄새...
Name: text, dtype: object
100     절단된 금속 냄새
101      가축 사료 냄새
102      제품 포장 냄새
103     포장지 처리 냄새
104     부패된 고기 냄새
105    공업용 윤활유 냄새
106     젖은 쓰레기 냄새
107     얼음이 녹은 냄새
108    플라스틱 태운 냄새
109     동물 배설물 냄새
Name: keyword, dtype: object


In [15]:
for text in val_smell_df.iloc[100:110]['text']:
    res = test_model(text, '/home/yjtech2/Desktop/yurim/LLM/Pre_processing/smell_keyword/smell_models/best_model_epoch_3', 'smell')
    print()
    print(text)
    print(res)


원래 문장:  여보세요? 유휴 토지 개발 현장에서 10개월 후 오후 10시부터 11시에 발생한 절단된 금속 냄새 건설 근로자의 건강 문제에 대해 상담 요청드립니다. 네, 유휴 토지 개발 현장에서 발생한 10개월 후 오후 10시부터 11시 절단된 금속 냄새 문제를 상담 드리겠습니다. 유휴 토지 개발 현장에서 10개월 후 오후 10시부터 11시에 절단된 금속 냄새으로 인해 호흡기 질환 문제가 발생하고 있습니다. 불편을 드려 정말 죄송합니다. 10개월 후 오후 10시부터 11시에 발생한 호흡기 질환 문제를 해결하기 위해 현장 점검을 진행 중입니다. 그럼 문제를 빨리 해결해 주세요. 확인 후 조치를 취하겠습니다. 수고 많으십니다. 저희가 도와드릴 수 있어 기쁩니다. 감사합니다.
smell키워드 추출: 한 절단된 금속 냄새

여보세요? 유휴 토지 개발 현장에서 10개월 후 오후 10시부터 11시에 발생한 절단된 금속 냄새 건설 근로자의 건강 문제에 대해 상담 요청드립니다. 네, 유휴 토지 개발 현장에서 발생한 10개월 후 오후 10시부터 11시 절단된 금속 냄새 문제를 상담 드리겠습니다. 유휴 토지 개발 현장에서 10개월 후 오후 10시부터 11시에 절단된 금속 냄새으로 인해 호흡기 질환 문제가 발생하고 있습니다. 불편을 드려 정말 죄송합니다. 10개월 후 오후 10시부터 11시에 발생한 호흡기 질환 문제를 해결하기 위해 현장 점검을 진행 중입니다. 그럼 문제를 빨리 해결해 주세요. 확인 후 조치를 취하겠습니다. 수고 많으십니다. 저희가 도와드릴 수 있어 기쁩니다. 감사합니다.
None
원래 문장:  저희 말 농장에서 가축 사료 냄새 문제로 불편을 겪고 있습니다. 30일 전 저녁 6시에 발생한 가축 사료 냄새과 관련해 문의 드립니다. 감사합니다. 농업 부서입니다. 30일 전 저녁 6시에 발생한 말 농장 문제를 도와드리겠습니다. 말 농장에서 30일 전 저녁 6시에 가축 사료 냄새으로 인해 호흡기 질환 문제가 발생하고 있습니다. 문제를 즉시 해결할 수 있도록 말 농장에 대

In [9]:
train_smell_df[train_smell_df['keyword'] != "고온 증기 냄새"]

,text,keyword
0,여보세요? 택배 물류 창고 근처에서 12개월 전 새벽 5시에 화장실 냄새 청결 부족...,화장실 냄새
1,안녕하세요. 지하철 역 주변에서 11월 14일 오전에 공공장소 쓰레기 냄새 관련하여...,공공장소 쓰레기 냄새
2,안녕하세요. 아파트 커뮤니티 센터에서 12월 8일 저녁 7시에 오래된 냄새 관련하여...,오래된 냄새
3,여보세요? 음식물 쓰레기장에서 1월 17일 자정에 발생한 썩은 물질 냄새 불법 폐기...,썩은 물질 냄새
4,여보세요? 돼지 농장에서 5월 4일 저녁 7시에 발생한 동물 건강 관리용 약물 냄새...,동물 건강 관리용 약물 냄새
...,...,...
23995,저희 에너지 플랜트 공사 현장에서 기타 건축 폐기물 냄새 문제로 불편을 겪고 있습니...,기타 건축 폐기물 냄새
23996,여보세요? 포항 근교 농촌에서 8월 2일 새벽에 발생한 여름 폭우 후 냄새 나는 호...,여름 폭우 후 냄새 나는 호수
23997,여보세요? 포항대학교에서 10월 4일 저녁 6시에 발생한 공공장소 쓰레기 냄새 도로...,공공장소 쓰레기 냄새
23998,축사에서 8월 1일 오후 8시부터 9시에 양식장에서 나는 냄새이 코끝을 찌르는 나고...,양식장에서 나는 냄새
